# Geographic Plots and Other Visualizations of The World's Historical Temperature Data Using Plotly and SQL

In this post, I will be creating some interesting plots using databases and plotly. We will be able to make visualizations previously not possible with the skills we learned in PIC16A. We're going to be working with data related to the temperatures observed in different countries around the world across the span of decades.

### Specifications

1. The query_climate_database() function is correctly defined according to the prompt.
2. There are two geographic scatterplots, one for 1980-2020 India that looks similar to the provided example, and another one for a different time and/or country.
3. The geographic scatterplots are correctly constructed and professionally labeled, including a title, hovers with rounded estimates, and a correctly centered colorbar.
4. There is at least one more sql query function defined.
5. There are two other interactive plots constructed using Plotly.
6. One of these plots involves the use of multiple facets.
7. The two other plots are wrapped in appropriate, user-friendly functions.
8. Each of the two other interactive plots have descriptive titles and centered color maps when appropriate.

## 1. Creating the Database

First, let's create a database that contains three tables: temperatures, stations, and countries

We will need to import some packages first. We import plotly.io as pio so that the figures in this notebook show up properly on the quarto blog post. We import sqlite3 to create the database, pandas and numpy for dataframe operations, and plotly to create our visualizations.

In [1]:
#Importing the packages
import plotly.io as pio
pio.renderers.default="iframe"
import sqlite3
import pandas as pd
import numpy as np
from plotly import express as px

Now, let's import the csv file for the list of countries, and the csv file for the stations in each country

In [2]:
#reading the temperature and country csv files we want to put into our database
countries_url = "https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"
countries = pd.read_csv(countries_url) #countries table
url = "C:/Users/nyc8p/PIC16B/Lecture/station-metadata.csv"
stations = pd.read_csv(url)
stations.head()

,ID,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,57.7667,11.8667,18.0,SAVE
1,AE000041196,25.3330,55.5170,34.0,SHARJAH_INTER_AIRP
2,AEM00041184,25.6170,55.9330,31.0,RAS_AL_KHAIMAH_INTE
3,AEM00041194,25.2550,55.3640,10.4,DUBAI_INTL
4,AEM00041216,24.4300,54.4700,3.0,ABU_DHABI_BATEEN_AIR


Taking a look at the countries dataframe

In [3]:
countries.head()

,FIPS 10-4,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania
3,AG,DZ,Algeria
4,AQ,AS,American Samoa


Now, let's make our database! We use sqlite3 to create the database. We will then add 3 tables to it, temperatures, ,countries, and stations. Let's add temperatures first

In [11]:
conn = sqlite3.connect("temps.db") # create a database in current directory called temps.db
df_iter = pd.read_csv("temps.csv", chunksize = 100000) #we use chunksize because temps.csv is a huge csv
df = df_iter.__next__() #point iterator to rows in the dataframe

In [12]:
df

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,ACW00011604,1961,-89.0,236.0,472.0,773.0,1128.0,1599.0,1570.0,1481.0,1413.0,1174.0,510.0,-39.0
1,ACW00011604,1962,113.0,85.0,-154.0,635.0,908.0,1381.0,1510.0,1393.0,1163.0,994.0,323.0,-126.0
2,ACW00011604,1963,-713.0,-553.0,-99.0,541.0,1224.0,1627.0,1620.0,1596.0,1332.0,940.0,566.0,-108.0
3,ACW00011604,1964,62.0,-85.0,55.0,738.0,1219.0,1442.0,1506.0,1557.0,1221.0,788.0,546.0,112.0
4,ACW00011604,1965,44.0,-105.0,38.0,590.0,987.0,1500.0,1487.0,1477.0,1377.0,974.0,31.0,-178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,BR000083361,1943,2553.0,2603.0,2533.0,2483.0,2473.0,2333.0,2283.0,2233.0,2523.0,2703.0,2663.0,2603.0
99996,BR000083361,1944,2733.0,2643.0,2623.0,2513.0,2453.0,2353.0,2253.0,2573.0,2783.0,2743.0,2573.0,2723.0
99997,BR000083361,1945,2643.0,2603.0,2603.0,2563.0,2183.0,2123.0,2293.0,2663.0,2743.0,2703.0,2633.0,2573.0
99998,BR000083361,1946,2663.0,2663.0,2653.0,2583.0,2483.0,2313.0,2223.0,2463.0,2743.0,2723.0,2813.0,2673.0


First, we need to do some data cleaning on temperatures.csv

In [13]:
def prepare_df(df):
    '''
    Clean df by stacking months and changing column names, returns cleaned df
    
    '''
    #create a new column of the country name id for later sql purposes
    df["FIPS 10-4"] = df["ID"].str[:2]
    df = pd.merge(df, countries, on="FIPS 10-4")
    
    #drop these unnecessary columns
    df = df.drop(["FIPS 10-4", "ISO 3166"], axis=1)
    
    df = df.set_index(keys=["ID", "Year", "Name"]) #we don't want to stack these columns
    df = df.stack() #stack df, converts it to a series
    df = df.reset_index() #convert it back to a dataframe 
    df = df.rename(columns = {"level_3"  : "Month" , 0 : "Temp"}) #change column names to be more proper
    # Make Month and Temp columns' data more appropriate
    df["Month"] = df["Month"].str[5:].astype(int) 
    df["Temp"]  = df["Temp"] / 100
    return(df)
df = prepare_df(df)

Now, we can add all of our csv files to the database! We do this with the to_sql() function. Here, we add temperatures to the database by iterating through df_iter

In [14]:
for df in df_iter: 
    df = prepare_df(df) #clean df
    df.to_sql("temperatures", conn, if_exists = "append", index = False)

Now, we add the rest of our csvs and check if we have 3 tables in our database

In [15]:
#Adding the csvs to the temps.db database
stations.to_sql("stations", conn, if_exists = 'append', index = False)
countries.to_sql("countries", conn, if_exists = 'append', index = False)
#Now, let's create a cursor object that we use to execute SQL commands on our database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())
conn.close()

[('temperatures',), ('stations',), ('countries',)]


Perfect! It looks like we have 3 tables in our database, temperatures, stations, and countries!

## 2. Writing an SQL query function to find out the yearly temperature increase of a certain country

We have a database, so now let's put it to use! Let's write a function called `query_climate_database()` that has 4 arguments:

- `country`, a string giving the name of a country for which data should be returned.
- `year_begin` and `year_end`, two integers giving the earliest and latest years for which should be returned.
- `month`, an integer giving the month of the year for which should be returned.

The function returns a pandas dataframe of temperature readings for the specified country in the specified year range and month of the year.

Dataframe's columns:

- station name
- The latitude of the station.
- The longitude of the station.
- The name of the country in which the station is located.
- The year in which the reading was taken.
- The month in which the reading was taken.
- The average temperature at the specified station during the specified year and month. 

In [16]:
def query_climate_database(country, year_begin, year_end, month):
    '''
    Takes string country (name of country), ints year_begin and year_end (year range), and int month (month of year) and returns a pandas 
    dataframe of temp readings for specified country in specified year range and month of the year using sql.
    '''
    conn = sqlite3.connect('temps.db') #connect to temps database
    cursor = conn.cursor() 
    
    #select the station name, latitude, longitude, country name, year, month, and temperature using our given arguments
    cmd = \
    f"""
    SELECT S.name, S.latitude, S.longitude, C.name, T.year, T.month, T.temp 
    FROM temperatures T 
    LEFT JOIN stations S on T.id = S.id
    LEFT JOIN countries C on T.name = C.name
    WHERE C.name = '{country}'
    AND T.year BETWEEN {year_begin} AND {year_end}
    AND T.month = {month};
    """
    df = pd.read_sql_query(cmd,conn) #read the query to a dataframe
    conn.close() #close connection to database
    return df
    

Now, let's test our function. Let's try to get a dataframe of India from 1980 - 2020 in the month of January

In [17]:
query_climate_database(country = "India", 
                       year_begin = 1980, 
                       year_end = 2020,
                       month = 1)

,NAME,LATITUDE,LONGITUDE,Name,Year,Month,Temp
0,PBO_ANANTAPUR,14.583,77.633,India,1980,1,23.48
1,PBO_ANANTAPUR,14.583,77.633,India,1981,1,24.57
2,PBO_ANANTAPUR,14.583,77.633,India,1982,1,24.19
3,PBO_ANANTAPUR,14.583,77.633,India,1983,1,23.51
4,PBO_ANANTAPUR,14.583,77.633,India,1984,1,24.81
...,...,...,...,...,...,...,...
3147,DARJEELING,27.050,88.270,India,1983,1,5.10
3148,DARJEELING,27.050,88.270,India,1986,1,6.90
3149,DARJEELING,27.050,88.270,India,1994,1,8.10
3150,DARJEELING,27.050,88.270,India,1995,1,5.60


Looks good!

## 3. Write a Geographic Scatter Function for Yearly Temperature Increases

Let's write a function `temperature_coefficient_plot()` to create visualizations that tell us how the average yearly change in temperature varies within a given country. The function takes in 5 arguments and an undetermined number of keyword arguments to help us with our visualizations:
- `country`, `year_begin`, `year_end`, and `month` should be as in the previous part.
- `min_obs`, the minimum required number of years of data for any given station. Only stations with at least `min_obs` years of data are plotted. Others are filtered out with df.transform()
- `**kwargs`, additional keyword arguments to pass to `px.scatter_mapbox()`
- The output is an interactive geographic scatterplot, constructed using Plotly Express, with a point for each station, such that the color of the point reflects an estimate of the yearly change in temperature during the specified month and time period at that station. 

First, in order to get the yearly change in temperature, we need the linear regression model's coefficient after modeling our dataframe.

In [18]:
from sklearn.linear_model import LinearRegression #import linear regression model

def coef(data_group):
    '''
    take data_group pandas dataframe and return the coefficient for linear regression 
    '''
    x = data_group[["Year"]] # 2 brackets because X should be a df
    y = data_group["Temp"]   # 1 bracket because y should be a series
    LR = LinearRegression()
    LR.fit(x, y)
    return LR.coef_[0]

Let's also define another data cleaning function in order to add estimated yearly increase of temperature to the dataframe after only choosing the stations that have at least min_obs years of data

In [19]:
def cleaning(df, min_obs):
    '''
    Filter pandas dataframe df so that it only has stations with at least min_obs years of data, then add estimated yearly temp increase column
    '''
    df["Yearcount"] = df.groupby('NAME')['Year'].transform('count') #add 'Yearcount' column: number of years of data for each station
    df = df[df["Yearcount"] > min_obs] #filter df by min_obs years
    
    #group by these columns and create a new column filled with yearly increase for each station
    coefs = df.groupby(["NAME","LATITUDE", "LONGITUDE", "Month"]).apply(coef).round(4) #rounding to 4 decimal places
    coefs = coefs.reset_index()
    coefs = coefs.rename(columns = {0: "Estimated Yearly Increase (\u00b0C)"}) #rename the column 
    return coefs

In [20]:
def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    '''
    Takes string country (name of country), ints year_begin and year_end (year range), int month (month of year), 
    int min_obs (number of years of data observed), and additional arguments to create a geographic scatterplot with 
    a point for each station where each point reflects the yearly change in temperature during the specified month and time period at that station
    '''
    df = query_climate_database(country, year_begin, year_end, month) #get dataframe using our function from before
    
    #change df so that we only have rows where stations have at least min_obs years of data
    df = cleaning(df, min_obs)
    #month dictionary for proper title of graph
    month_dict = {
        1: 'January',
        2: 'February',
        3: 'March',
        4: 'April',
        5: 'May',
        6: 'June',
        7: 'July',
        8: 'August',
        9: 'September',
        10: 'October',
        11: 'November',
        12: 'December'
    }
    #create plot with proper parameters 
    return px.scatter_mapbox(df,
                       lat = df["LATITUDE"],
                       lon = df["LONGITUDE"],
                       color = df["Estimated Yearly Increase (\u00b0C)"],
                       hover_name = df["NAME"],
                      title = f"Estimates of yearly increase in temperature in {month_dict[month]} " \
                             f"for stations in {country}, years {year_begin} - {year_end}",
                             color_continuous_midpoint = 0, #sets colorbar midpoint at 0
                       **kwargs
                     )
    

Let's test our function. We can specify a color map first, then input our parameters for our function. Let's make a geographic scatterplot of India's 

In [21]:
color_map = px.colors.diverging.balance # choose a colormap

fig = temperature_coefficient_plot("India", 1980, 2020, 10, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map
                                  )
fig.show()

And there's our geographic scatterplot! We can see from this plot that India's temperature has steadily increased in October since 1980 due to the sheer amount of red points compared to the greyer points.

## 4. Two More Interesting Figures

Now, let's answer 2 more questions with some plotly visulizations and SQL queries.

One question that we can address is:

#### Question 1: In the United States, which stations had the largest change of temperature in December from 2000 to 2020?

First, we need to write an SQL query to get a proper dataframe to work with.

In [74]:
#idea: let's create geo scatterplot and have point size correlate with change of temperature
def query1():
    '''
    runs sql query to return a pandas dataframe of the US' data in December from 2000 and 2020 
    '''
    conn = sqlite3.connect('temps.db') #connect to temps database
    cursor = conn.cursor() 
    
    #select the station name, latitude, longitude, country name, year, month, and temperature using our given arguments
    cmd = \
    f"""
    SELECT S.name, S.latitude, S.longitude, C.name, T.year, T.month, T.temp 
    FROM temperatures T 
    LEFT JOIN stations S on T.id = S.id
    LEFT JOIN countries C on T.name = C.name
    WHERE (T.year = 2000 OR T.year = 2020) AND T.month = 12 AND C.name = 'United States';
    """
    df = pd.read_sql_query(cmd,conn) #read the query to a dataframe
    conn.close() #close connection to database
    return df

df1 = query1()
df1

,NAME,LATITUDE,LONGITUDE,Name,Year,Month,Temp
0,ADDISON,34.2553,-87.1814,United States,2020,12,4.70
1,ALEXANDER_CITY,32.9453,-85.9481,United States,2000,12,3.86
2,ALEXANDER_CITY,32.9453,-85.9481,United States,2020,12,7.37
3,ALICEVILLE,33.1272,-88.1550,United States,2020,12,7.07
4,ANDALUSIA_3_W,31.3072,-86.5225,United States,2000,12,7.03
...,...,...,...,...,...,...,...
12354,SISSETON_MUNI_AP,45.6689,-96.9914,United States,2020,12,-3.43
12355,BOSCOBEL_AP,43.1561,-90.6775,United States,2000,12,-11.62
12356,BOSCOBEL_AP,43.1561,-90.6775,United States,2020,12,-2.62
12357,LINCOLN_8_ENE,40.8483,-96.5650,United States,2020,12,-1.32


Now, let's clean this dataframe. We can count up the amount of times each station appears in the dataframe. If the count isn't 2, then that means the station doesn't have data for either 2000 or 2020 and can be removed. We also add a new column containing the total temperature change from 2000 to 2020.

We need a function to put in apply() later where we can subtract the temperature column's entries appropriately.

In [148]:
def difference(data):
    '''
    Take pandas dataframe data and subtract 2000 temperature from 2020 temperature and make that a new column. return copy of data 
    '''
    result = data.copy()
    result["Temperature Change (\u00b0C)"] = result["Temp"].iloc[1] - result["Temp"].iloc[0]
    return result

Onto the full data cleaning function:

In [155]:
def cleaning1(df):
    '''
    Takes pandas dataframe df and counts up number of times station appears. Only include stations that appear twice.
    Add column called temp changereturns edited pandas dataframe
    '''
    df["count"] = df.groupby('NAME')['NAME'].transform('count') #add count column
    df = df[df['count'] == 2] #only include count = 2
    df = df.drop("count", axis = 1)  #drop the count now
    #sort by alphabetical order
    df = df.sort_values("NAME")
    df = df.reset_index(drop=True)
    
    #groupby name and apply difference function
    df = df.groupby('NAME').apply(difference)
    df = df[df["Year"] == 2020] #get rid of half the rows
    df = df.drop(["Year", "Temp"], axis = 1).reset_index(drop = True) #drop some rows
    df = df[df["Temperature Change (\u00b0C)"] >= 0]
    return df

df = cleaning1(df1)
df

,NAME,LATITUDE,LONGITUDE,Name,Month,Temperature Change (°C)
0,ABERDEEN_EXP_STN,42.9536,-112.8253,United States,12,1.43
1,ABERNATHY_MOUNTAIN_WASHINGTON,46.3500,-123.1000,United States,12,1.06
3,ABINGDON_3S,36.6686,-81.9647,United States,12,4.51
4,ABIQUIU_DAM,36.2403,-106.4278,United States,12,2.02
7,ADDISON,34.2553,-87.1814,United States,12,10.38
...,...,...,...,...,...,...
3965,ZANESVILLE_MUNI_AP,39.9444,-81.8922,United States,12,5.77
3966,ZIM_OREGON,42.6850,-122.3903,United States,12,1.23
3968,ZORTMAN,47.9186,-108.5244,United States,12,8.76
3969,ZUMBROTA,44.2992,-92.6661,United States,12,9.78


Now, let's make a geographic scatterplot using this data! This time, instead of differentiating the points based on color, we can do it by size of the point! So larger datapoints would indicate a bigger change in temperature

In [159]:
fig = px.scatter_mapbox(df,
                        lat = df["LATITUDE"],
                        lon = df["LONGITUDE"],
                        hover_name = df["NAME"],
                        title = "Relative Size of Changes in Temperature from 2000 to 2020 in December across the United States",
                        zoom = 3,
                        mapbox_style="carto-positron",
                        size = df["Temperature Change (\u00b0C)"]
                       )
fig.show()
                  

So from this plot, it looks like the biggest points are in the middle and east coast of the United States. Areas on the west coast in comparison have very small points, indicating that their changes in temperature during December have not been as severe as the changes occuring in the eastern half of the country.

#### Question 2: Between the United States and Canada, of their respective stations with the heighest change of temperature in December from 2000 to 2020, which country had a bigger change in temperature?

Define our SQL query for this question:

In [161]:
#idea: create lineplot with facet = country 
def query2():
    '''
    runs sql query to return a pandas dataframe of countries' data in December from 2000-2020 
    '''
    conn = sqlite3.connect('temps.db') #connect to temps database
    cursor = conn.cursor() 
    
    #select the station name, latitude, longitude, country name, year, month, and temperature using our given arguments
    cmd = \
    f"""
    SELECT S.name, S.latitude, S.longitude, C.name, T.year, T.month, T.temp 
    FROM temperatures T 
    LEFT JOIN stations S on T.id = S.id
    LEFT JOIN countries C on T.name = C.name
    WHERE (T.year BETWEEN 2000 AND 2020) AND (T.month = 12) AND (C.name = 'United States' OR C.NAME = 'Canada');
    """
    df = pd.read_sql_query(cmd,conn) #read the query to a dataframe
    conn.close() #close connection to database
    return df

df = query2()
df

,NAME,LATITUDE,LONGITUDE,Name,Year,Month,Temp
0,CHEMAINUS,48.9333,-123.7500,Canada,2000,12,3.56
1,CHEMAINUS,48.9333,-123.7500,Canada,2001,12,3.79
2,CHEMAINUS,48.9333,-123.7500,Canada,2002,12,5.27
3,CHEMAINUS,48.9333,-123.7500,Canada,2003,12,4.50
4,CHEMAINUS,48.9333,-123.7500,Canada,2004,12,5.48
...,...,...,...,...,...,...,...
166205,LINCOLN_11_SW,40.6953,-96.8542,United States,2016,12,-3.04
166206,LINCOLN_11_SW,40.6953,-96.8542,United States,2017,12,-2.33
166207,LINCOLN_11_SW,40.6953,-96.8542,United States,2018,12,-0.59
166208,LINCOLN_11_SW,40.6953,-96.8542,United States,2019,12,0.41


Now, let's clean our data, getting just the 1 station from each country that had the biggest change in temperature from 2000 to 2020